In [ ]:
try:
    import pymc as pm
except:
    %pip install pymc==5.8.0    
    import pymc as pm
    
print(pm.__version__)

try:
    import pymc_bart as pmb
except:
    %pip install pymc-bart
    import pymc_bart as pmb
    
try:
    import cloudpickle
except:
    %pip install cloudpickle
    import cloudpickle
    
try:
    import pymc_experimental
except:
    %pip install pymc-experimental==0.0.11
    import pymc_experimental
    
print(cloudpickle.__version__)

%matplotlib inline
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import boto3
from sagemaker import get_execution_role
import os
import pickle
import xgboost

from sklearn.metrics import confusion_matrix

import sys
from sagemaker.session import Session

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

In [ ]:
# read inputs
# code below is to get data from s3 bucket...?
role = get_execution_role() # not necessary
region = boto3.Session().region_name # not necessary
bucket='us.com.syngenta.ap.nonprod' # Replace with your s3 bucket name

df_all_melt = pd.DataFrame()
for yr in ['2020','2021','2022']:
    fpath = 'uat/dme/performance/compute_pred_adv_data_collected/data/SOY_NA_SUMMER/SingleExp/{}'.format(yr) # fpath within bucket
    fname = 'pred_adv_data_collected.csv' # filename
    data_location = 's3://{}/{}/{}'.format(bucket,fpath,fname)
    df_temp = pd.read_csv(data_location)
    df_all_melt = pd.concat((df_all_melt,df_temp),axis=0)

In [ ]:
# pivot table, get appropriate stage
# set index columns previously used to melt data
melt_cols = ['ap_data_sector','analysis_year','entry_identifier',\
             'material_type','decision_group_rm','technology','trait','var']

#if DKU_DST_ap_data_sector == 'SOY_NA_SUMMER':
    #melt_cols.remove('source_id')

melt_no_stack_cols = melt_cols.copy()
melt_no_stack_cols.remove('trait')
melt_no_stack_cols.remove('var')

# set some inputs and clean up some columns
if DKU_DST_ap_data_sector == 'CORN_NA_SUMMER' or DKU_DST_ap_data_sector == 'SOY_NA_SUMMER':
    yield_trait = 'YGSMN'
elif DKU_DST_ap_data_sector == 'CORN_EAME_SUMMER':
    yield_trait = 'YSDMN'

df_all_melt['technology'] = df_all_melt['technology'].fillna(value='conv')

# update type of some cols
float_cols = ['stage','decision_group_rm']
for col in float_cols:
    df_all_melt[col] = df_all_melt[col].astype(float)

# replace Nan's for some cols
df_all_melt['trait'][df_all_melt['trait'].isna()] = ''
#df_all_melt['decision_group'][df_all_melt['decision_group'].isna()] = 'na'
df_all_melt[melt_no_stack_cols] = df_all_melt[melt_no_stack_cols].fillna(value=-1045)


# fix strings to other tpyes
for as_str, as_bool in zip(['False','True'],[0, 1]):
    df_all_melt['value'][df_all_melt['value'] == as_str] = as_bool

# shorten some variable names
var_mapper = {'result_numeric_value':'result'}
for var in var_mapper:
    df_all_melt['var'][df_all_melt['var']==var]=var_mapper[var]

In [ ]:
### condense df_all_melt to avoid memory issues when pivoting
df_all_melt = df_all_melt[(df_all_melt['var'] == 'result_diff') | (df_all_melt['var']=='current_stage') | (df_all_melt['var'] == 'was_adv')]

In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# pivot dataframe by trait and var (only care about current year's data for now)
#df_all_melt['value'] = df_all_melt['value'].astype(float)

df_all = df_all_melt.pivot_table(values=['value'],index=melt_no_stack_cols,columns=['var','trait'],
                                      aggfunc={'value':'first'}).reset_index()
# rename columns and reset index
df_all.columns = ['_'.join(filter(None,col_tuple)).replace('value','').replace('alpha_','').strip('_') for col_tuple in df_all.columns]
df_all = df_all.rename_axis(None, axis=1)

# update datetype for certain columns
type_map = {'prediction':float, 'stderr':float, 'result':float,
            'result_diff':float,'rm_estimate':float, 'e_rm':float,
            'bvaln':float, 'sbssn':float,
            'prev_stage':float, 'current_stage':float,'next_stage':float,'third_stage':float,'selection_remark':float,
            'was_adv':bool,'was_adv_next':bool}

for key in type_map.keys():
    cols_to_update = list(df_all.columns[[key in col for col in df_all.columns]])
    df_all[cols_to_update] = df_all[cols_to_update].astype(type_map[key])

In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
def process_harvt(row_in):
    if isinstance(row_in,str):
        return 'drop' in row_in.lower()
    else:
        return False

#### FILTER TO DESIRED DATA for training
# filter materials: want stages 1-4, material type is parent for corn
# want stages 2-5, material type is entry for soy
if DKU_DST_ap_data_sector == "CORN_NA_SUMMER":
    df_piv = df_all[(df_all['material_type']!='entry') &
                       (df_all['current_stage'] <= 4) &
                       (df_all['current_stage'] >= 2)]

    # drop edge decision groups when training
    df_piv = df_piv[(df_piv['decision_group_rm'] > 80) & (df_piv['decision_group_rm'] < 120)]

    # drop any materials that had a selection_remark == drop for training
    df_piv = df_piv[df_piv['selection_remark'] != 9]

elif DKU_DST_ap_data_sector == 'SOY_NA_SUMMER':
    #df_all['harvt_drop'] = df_all['harvt'].apply(process_harvt)
    df_all['harvt_drop'] = False
    df_piv = df_all[(df_all['material_type']=='entry') &
                      (df_all['current_stage'] >= 3) &
                      (df_all['current_stage'] <= 3) &
                       (df_all['harvt_drop']==False)]

    #### preprocess text traits for soy
    extra_traits = ['decision_group_rm']
    numeric_raw_traits = ['PLHTN','IC__N','MRTYN']
    numeric_diff_traits = ['EMRGR',\
                     'SBSSN','YGSMN','HLDGR','FL_CR','GLU_R','HVAPR',\
                     'MI__R','PD_CR','RUR_R']
    text_traits = ['bp_t','bsr_t','cls_t','cn3_t','dic_t',\
                  'dpm_t','e1_t','e3_t','fels_t','fl_ct','ll55_t',\
                  'met_t','mi__t','pb_ct','rr2_t',\
                  'stmtt','sts_t','hilct' ,'rps_t']

    text_suffix = {'bsr_t':'Rbs1','fels_t':'Rcs3'}
    text_prefix = {'fels_t':'','bp_t':'','bsr_t':'','cls_t':'','dic_t':'','dpm_t':'',
                    'e1_t':'','e3_t':'','fels_t':'','fl_ct':'','ll55_t':'','met_t':'',
                    'mi__t':'','pb_ct':'','pd_ct':'','rr2_t':'','stmtt':'','sts_t':''}

    clean_text = {'cls_t':'_'}

In [ ]:
# set train and test data



In [ ]:
# train xgb model
xgb_params = {'max_depth':5,
              'reg_lambda':100,
              'subsample':0.2,
              'learning_rate':0.005,
              'n_estimators':500,
             'scale_pos_weight':10,
             'gamma':10,
             'booster':'gbtree'}

xgb_mdl = xgboost.XGBClassifier(**xgb_params)
xgb_mdl.fit(x_tr,y_tr)

In [ ]:
# test xgb model
y_proba = xgb_mdl.predict_proba(x_te)[:,1]

plt.figure()
plt.hist(y_proba[y_te==0],bins=100,histtype='step',density=True)
plt.hist(y_proba[y_te==1],bins=100,histtype='step',density=True)

print(confusion_matrix(y_true=y_te, y_pred=y_proba>np.percentile(y_proba,80)))

In [ ]:
# train bart model
with pm.Model() as bart_model:
    X = pm.MutableData("X", x_tr)
    Y = y_tr
    mu = pmb.BART("mu", X=X, Y=Y, m=20)
    p = pm.Deterministic('p', pm.math.invlogit(mu))
    pm.Bernoulli("obs", p=p, observed=Y, shape=mu.shape)#shape=p.shape)
    #obs = pm.Categorical("obs", p=theta.T, observed=Y)
    idata_train = pm.sample(tune=1,draws=1,chains=1)
    posterior_predictive_train = pm.sample_posterior_predictive(trace=idata_train,
                                                               var_names=["p","obs"])
    
# training dataset validation
prior_train = posterior_predictive_train.posterior_predictive.p.mean(axis=1).T

plt.hist(prior_train[y_tr==0],histtype='step',bins=100,density=True);
plt.hist(prior_train[y_tr==1],histtype='step',bins=100,density=True);
print(confusion_matrix(y_true=y_tr, y_pred=prior_train>np.percentile(prior_train,80)))

In [ ]:
# save model, trace, training data
with open("BART_test_sample2.pkl", 'wb') as f:
    cloudpickle.dump({'model':bart_model, 'trace':idata_train,'x_tr':x_tr,'y_tr':y_tr},f)

In [ ]:
# pickle.load?



In [ ]:
# load in model, test on test data

#with open("BART_test_sample.pkl",'rb') as buff:
with open("BART_test_sample.pkl",'rb') as buff:
    data = cloudpickle.load(buff)
    
with data['model']:
    pm.set_data({"X":x_te})
    posterior_predictive_test = pm.sample_posterior_predictive(trace=data['trace'],
                                                              var_names=["p","obs"])

In [ ]:
# test validation
prior_test = posterior_predictive_test.posterior_predictive.p.mean(axis=1).T

plt.hist(prior_test[y_te==0],histtype='step',bins=100,density=True);
plt.hist(prior_test[y_te==1],histtype='step',bins=100,density=True);
print(confusion_matrix(y_true=y_te, y_pred=prior_test>np.percentile(prior_test,80)))

In [ ]:
# things to do: 
# plot P against YGSMN, HLDGR, etc. to make sure output makes some sense
# variable importance plots
# Validate credibility interval
# test how # of tune-in affects output (read last N p-values, look at confusion matrices)

In [ ]:
# code below is a simple test of BART

In [ ]:
fname = 'BART_test_sample2.pkl' # filename

n1 = 1050 # n in each class
n2 = 150
n_dim = 10

mean1 = 1
std1 = 1
mean2 = 2
std2 = 1

x1 = np.random.randn(n1,n_dim)*std1 + mean1
y1 = np.zeros((n1,))
x2 = np.random.randn(n2,n_dim)*std2 + mean2
y2 = np.ones((n2,))

x_in = np.concatenate((x1,x2),axis=0)
y_in = np.concatenate((y1,y2),axis=0)

x_tr, x_te, y_tr, y_te = train_test_split(x_in, y_in, test_size=0.33)

plt.figure()
plt.hist(x1[:,0],bins=20,histtype='step',density=True);
plt.hist(x2[:,0],bins=20,histtype='step',density=True);

In [ ]:
# define and train model
with pm.Model() as bart_model:
    X = pm.MutableData("X", x_tr, dims=("obs","feature"))
    Y = y_tr
    mu = pmb.BART("mu", X=X, Y=Y, m=10, dims="obs", shape=X.shape[0])
    p = pm.Deterministic('p', pm.math.invlogit(mu), dims="obs")
    pm.Bernoulli("y", p=p, observed=Y, dims="obs",shape=p.shape[0])#shape=p.shape)
    idata_train = pm.sample(tune=2,draws=10,chains=1)

In [ ]:
tree = pmb.BART.all_trees[0][0][0]
tree.predict(x_te)

In [ ]:
with open("tree_test.pkl", 'wb') as f:
    pickle.dump({'tree':tree},f)

In [ ]:
with open("tree_test.pkl", 'rb') as f:
    data = pickle.load(f)

In [ ]:
data['tree'].predict(x_te)

In [ ]:
tree_out = pmb.utils._sample_posterior(pmb.BART.all_trees, x_te, rng=np.random.default_rng())
tree_out.shape

In [ ]:
trees = pmb.BART.all_trees
with open("BART_trees.pkl", 'wb') as f:
    pickle.dump({'all_trees':trees},f)

In [ ]:
with open("BART_trees.pkl", 'rb') as f:
    data = pickle.load(f)
    
tree_out = pmb.utils._sample_posterior(data['all_trees'], x_te, rng=np.random.default_rng())

In [ ]:
with new_model:
    pm.set_data({"X":x_te})
    posterior_predictive_test = pm.sample_posterior_predictive(trace=idata_train,
                                                              var_names=["p"])
    
prior_test = posterior_predictive_test.posterior_predictive.p.mean(axis=1).T

plt.figure()
plt.hist(prior_test[y_te==0],histtype='step',bins=20,density=True);
plt.hist(prior_test[y_te==1],histtype='step',bins=20,density=True);

In [ ]:
with pm.Model() as bart_model:
    X = pm.MutableData("X", x_tr, dims=("obs","feature"))
    Y = y_tr
    mu = pmb.BART("mu", X=X, Y=Y, m=10, dims="obs", shape=X.shape[0])
    p = pm.Deterministic('p', pm.math.invlogit(mu), dims="obs")
    pm.Bernoulli("y", p=p, observed=Y, dims="obs",shape=p.shape[0])#shape=p.shape)
    idata_train = pm.sample(tune=2,draws=10,chains=1)

pickled_mdl = pickle.dumps({'model':bart_model,'trace':idata_train,'x_te':x_te})

In [ ]:
# save model
pickled_mdl = cloudpickle.dumps({'model':bart_model,'trace':idata_train,'x_te':x_te})

with open("BART_model.pkl",'wb') as f:
    pickle.dump(pickled_mdl, f)

In [ ]:
with open("BART_model.pkl",'rb') as f:
    loaded_pickled = pickle.load(f)  
data = cloudpickle.loads(loaded_pickled)
    
with data['model']:
    pm.set_data({"X":data['x_te']})
    posterior_predictive_test = pm.sample_posterior_predictive(trace=data['trace'],
                                                              var_names=["p","y"])
    
prior_test = posterior_predictive_test.posterior_predictive.p.mean(axis=1).T

plt.figure()
plt.hist(prior_test[y_te==0],histtype='step',bins=20,density=True);
plt.hist(prior_test[y_te==1],histtype='step',bins=20,density=True);
    

In [ ]:
fname = 'BART_test_sample2.pkl' # filename
s3_fpath = 'uat/dme/performance/compute_model_preprocessor'
# upload to s3
bucket='us.com.syngenta.ap.nonprod' 

n1 = 1050 # n in each class
n2 = 150
n_dim = 10

mean1 = 1
std1 = 1
mean2 = 2
std2 = 1

x1 = np.random.randn(n1,n_dim)*std1 + mean1
y1 = np.zeros((n1,))
x2 = np.random.randn(n2,n_dim)*std2 + mean2
y2 = np.ones((n2,))

x_in = np.concatenate((x1,x2),axis=0)
y_in = np.concatenate((y1,y2),axis=0)

x_tr, x_te, y_tr, y_te = train_test_split(x_in, y_in, test_size=0.33)

plt.figure()
plt.hist(x1[:,0],bins=20,histtype='step',density=True);
plt.hist(x2[:,0],bins=20,histtype='step',density=True);

# define and train model
with pm.Model() as bart_model:
    X = pm.MutableData("X", x_tr)
    Y = y_tr
    mu = pmb.BART("mu", X=X, Y=Y, m=10, shape=X.shape[0])
    p = pm.Deterministic('p', pm.math.invlogit(mu))
    pm.Bernoulli("obs", p=p, observed=Y, shape=X.shape[0])#shape=p.shape)
    idata_train = pm.sample(tune=10,draws=50,chains=1)
    
with pm.Model() as bart_model2:
    X = pm.MutableData("X", x_tr)
    Y = y_tr
    mu = pmb.BART("mu", X=X, Y=Y, m=2, shape=X.shape[0])
    p = pm.Deterministic('p', pm.math.invlogit(mu))
    pm.Bernoulli("obs", p=p, observed=Y, shape=X.shape[0])#shape=p.shape)
    idata_garbage = pm.sample(tune=0,draws=1,chains=1)

In [ ]:
with bart_model:
    pm.set_data({"X":x_te})
    posterior_predictive_test = pm.sample_posterior_predictive(trace=idata_train,
                                                              var_names=["p","obs"])
    
prior_test = posterior_predictive_test.posterior_predictive.p.mean(axis=1).T

plt.figure()
plt.hist(prior_test[y_te==0],histtype='step',bins=20,density=True);
plt.hist(prior_test[y_te==1],histtype='step',bins=20,density=True);

In [ ]:
from typing import Dict, List, Optional, Tuple, Union

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr

from numpy.random import RandomState

from pymc_experimental.model_builder import ModelBuilder


%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 8927

rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [ ]:
# Generate data
n1 = 1050 # n in each class
n2 = 150
n_dim = 10

mean1 = 1
std1 = 1
mean2 = 2
std2 = 1

x1 = np.random.randn(n1,n_dim)*std1 + mean1
y1 = np.zeros((n1,))
x2 = np.random.randn(n2,n_dim)*std2 + mean2
y2 = np.ones((n2,))

x_in = np.concatenate((x1,x2),axis=0)
y_in = np.concatenate((y1,y2),axis=0)

x_tr, x_te, y_tr, y_te = train_test_split(x_in, y_in, test_size=0.33)

In [ ]:
class BARTModel(ModelBuilder):
    # Give the model a name
    _model_type = "BartModel"

    # And a version
    version = "0.1.1"

    def build_model(self, X: pd.DataFrame, y: Union[pd.Series, np.ndarray], **kwargs):
        """
        build_model creates the PyMC model

        Parameters:
        model_config: dictionary
            it is a dictionary with all the parameters that we need in our model example:  a_loc, a_scale, b_loc
        data: Dict[str, Union[np.ndarray, pd.DataFrame, pd.Series]]
            Data we want our model fit on.
        """
        # Check the type of X and y and adjust access accordingly
        X_values = X.values
        y_values = y.values if isinstance(y, pd.Series) else y
        self._generate_and_preprocess_model_data(X_values, y_values)

        # define and train model
        with pm.Model() as bart_model:
            X = pm.MutableData("X", x_tr, dims=("obs","feature"))
            Y = y_tr
            mu = pmb.BART("mu", X=X, Y=Y, m=10, dims="obs", shape=X.shape[0])
            p = pm.Deterministic('p', pm.math.invlogit(mu), dims="obs")
            pm.Bernoulli("y", p=p, observed=Y, dims="obs",shape=p.shape[0])#shape=p.shape)
            idata_train = pm.sample(tune=2,draws=10,chains=1)
        
        with pm.Model() as self.model:
            # Create mutable data containers
            x_data = pm.MutableData("x_data", X_values, dims=("obs", "feature"))
            y_data = y_values
            
            # BART model and bernoulli output
            mu = pmb.BART("mu", X=x_data, Y=y_data, m=10, dims="obs", shape=x_data.shape[0])
            p = pm.Deterministic('p', pm.math.invlogit(mu), dims="obs")
            obs = pm.Bernoulli("y", p=p, observed=y_data, shape=p.shape[0])#shape=x_data.shape)

    def _data_setter(
        self, X: Union[pd.DataFrame, np.ndarray], y: Union[pd.Series, np.ndarray] = None
    ):
        if isinstance(X, pd.DataFrame):
            x_values = X.values
        else:
            # Assuming "input" is the first column
            x_values = X

        with self.model:
            pm.set_data({"x_data": x_values})
            if y is not None:
                pm.set_data({"y_data": y.values if isinstance(y, pd.Series) else y})

    @property
    def default_model_config(self) -> Dict:
        """
        default_model_config is a property that returns a dictionary with all the prior values we want to build the model with.
        It supports more complex data structures like lists, dictionaries, etc.
        It will be passed to the class instance on initialization, in case the user doesn't provide any model_config of their own.
        """
        model_config: Dict = {
        }
        return model_config

    @property
    def default_sampler_config(self) -> Dict:
        """
        default_sampler_config is a property that returns a dictionary with all most important sampler parameters.
        It will be used in case the user doesn't provide any sampler_config of their own.
        """
        sampler_config: Dict = {
            "draws": 2,
            "tune": 1,
            "chains": 1
        }
        return sampler_config

    @property
    def output_var(self):
        return "p"

    @property
    def _serializable_model_config(self) -> Dict[str, Union[int, float, Dict]]:
        """
        _serializable_model_config is a property that returns a dictionary with all the model parameters that we want to save.
        as some of the data structures are not json serializable, we need to convert them to json serializable objects.
        Some models will need them, others can just define them to return the model_config.
        """
        return self.model_config

    def _save_input_params(self, idata) -> None:
        """
        Saves any additional model parameters (other than the dataset) to the idata object.

        These parameters are stored within `idata.attrs` using keys that correspond to the parameter names.
        If you don't need to store any extra parameters, you can leave this method unimplemented.

        Example:
            For saving customer IDs provided as an 'customer_ids' input to the model:
            self.customer_ids = customer_ids.values #this line is done outside of the function, preferably at the initialization of the model object.
            idata.attrs["customer_ids"] = json.dumps(self.customer_ids.tolist())  # Convert numpy array to a JSON-serializable list.
        """
        pass

        pass

    def _generate_and_preprocess_model_data(
        self, X: Union[pd.DataFrame, pd.Series], y: Union[pd.Series, np.ndarray]
    ) -> None:
        """
        Depending on the model, we might need to preprocess the data before fitting the model.
        all required preprocessing and conditional assignments should be defined here.
        """
        self.model_coords = None  # in our case we're not using coords, but if we were, we would define them here, or later on in the function, if extracting them from the data.
        # as we don't do any data preprocessing, we just assign the data givenin by the user. Note that it's very basic model,
        # and usually we would need to do some preprocessing, or generate the coords from the data.
        self.X = X
        self.y = y

In [ ]:
#X = pd.DataFrame(data=x_tr)
#y = y_tr
X = pd.DataFrame(data=x_tr, columns=["input"+str(i) for i in range(x_tr.shape[1])])
y = y_tr.copy()

model = BARTModel()
idata = model.fit(X, y)

In [ ]:
fname = "BART_model_v0.nc"
model.save(fname)

In [ ]:
model_2 = BARTModel.load(fname)

In [ ]:
x_pred = x_te
prediction_data = pd.DataFrame(data=x_pred, columns=["input"+str(i) for i in range(x_te.shape[1])])
type(prediction_data.values)

In [ ]:
with model_2.model:
    pm.set_data({"x_data":x_te})
    #posterior_predictive_test = pm.sample_posterior_predictive(trace=model_2.idata,
    #                                                          var_names=["p"])
    prior_predictive_test = pm.sample_prior_predictive(var_names=["p"])

In [ ]:
prior_test = posterior_predictive_test.posterior_predictive.p.mean(axis=1).values.reshape((-1,1))

plt.hist(prior_test[y_tr==0],density=True,bins=20,histtype='step');
plt.hist(prior_test[y_tr==1],density=True,bins=20,histtype='step');

In [ ]:
pred_mean = model_2.predict(prediction_data)
# samples
pred_samples = model_2.predict_posterior(prediction_data)

In [ ]:
plt.hist(pred_mean[y_te==0],density=True,bins=20,histtype='step');
plt.hist(pred_mean[y_te==1],density=True,bins=20,histtype='step');

In [ ]:
plt.plot(x_te[:,1], np.std(pred_samples,axis=1),'.')